In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format('delta').load('abfss://bronze@dbxnetflix.dfs.core.windows.net/netflix_titles')

In [0]:
df.display()

In [0]:
df = df.drop('_rescued_data')

In [0]:
df.display()

In [0]:
df = df.dropna(subset=['type'])

In [0]:
df.display()

In [0]:
df = df.dropna(subset=['rating', 'description', 'show_id'])

In [0]:
df.display()

In [0]:
df = df.fillna({'duration_minutes': 0, 'duration_seasons': 0})

df.display()

In [0]:
df = df.withColumn('duration_minutes', col('duration_minutes').cast(IntegerType()))\
    .withColumn('duration_seasons', col('duration_seasons').cast(IntegerType()))

In [0]:
df.display()

In [0]:
df = df.withColumn('short_title', split(col('title'), ':').getItem(0))
df.display()

In [0]:
df = df.withColumn('flag', when(col('type') == 'Movie', 0).when(col("type") == 'TV Show', 1).otherwise(2))
df.display()

In [0]:
windowspec = Window.orderBy(col('duration_minutes').desc())
df.withColumn('rank', dense_rank().over(windowspec))
df.display()

In [0]:
df.groupBy('type').agg(count('*').alias('tot')).display()

In [0]:
df = df.withColumn('release_year', col("release_year").cast(IntegerType()))\
        .withColumn('date_added', to_date(col("date_added"), 'M/d/yyyy'))

In [0]:
df.display()

In [0]:
df = df.withColumn('show_id_len', length('show_id'))
df.display()

In [0]:
df_valid = df.filter(col('show_id').rlike('^[0-9]+$'))
df_valid.display()

In [0]:
df_valid = df_valid.drop('show_id_len')

In [0]:
df_valid = df_valid.withColumn('show_id', col('show_id').cast(IntegerType()))


In [0]:
df_valid = df_valid.withColumn('title', regexp_replace(col('title'), r'[#"\'()]+', ""))\
                    .withColumn('title', initcap('title'))



In [0]:
df_valid.display()

In [0]:

df_valid.write.format('delta')\
    .mode('overwrite')\
    .option('path', 'abfss://silver@dbxnetflix.dfs.core.windows.net/netflix_titles')\
    .save()
